In [1]:
#I need to quantify diversity now

In [11]:
import glob
import os
import pandas as pd
import json
import numpy as np
import gzip
from collections import defaultdict

# Things that should be packages

In [3]:
# from collections.abc import Iterable, MutableSequence
# import os
# import numpy as np

# class SimplePDBParser:
#     def __init__(self,data_input=None):
#         self.records = []

#         #must determine the nature of the data_input
#         if type(data_input) == None:
#             #nothing to do...
#             pass
#         elif type(data_input) == Iterable:
#             self.parse_stream(data_input)
#         elif type(data_input) == str:
#             #this could be a number of things
#             if os.path.isfile(data_input):
#                 self.parse_file(data_input)
#             else:
#                 self.parse_string(data_input)


#     #returns the number of ATOM records with unique chainID and resSeq
#     def __len__(self) -> int:
#         unique_reses = set()
#         for record in self.records:
#             if record.record_type == "ATOM":
#                 unique_reses.add(record.chainID+str(record.resSeq))
#         return len(unique_reses)

#     #collects all xyz coordinates of all ATOM records and returns them in an array in the order they appear in the pdbs
#     def coords(self) -> np.array:
#         data = []
#         for record in self.records:
#             if record.record_type == "ATOM":
#                 data.append((record.x,record.y,record.z))
#         return np.array(data)

#     def __str__(self):
#         return "\n".join([str(record) for record in self.records])

#     #returns an npose based on this pdb file
#     def npose(self):
#         raise NotImplementedError()


#     def extract_chain(self,chainID,preserve_other_records=True):
#         new_pdb = SimplePDBParser()
#         new_records = []
#         for record in self.records:
#             if preserve_other_records:
#                 if record.record_type != "ATOM" or record.chainID == chainID:
#                     new_records.append(record)

#         new_pdb.records = new_records

#         return new_pdb

#     class BasicRecord:
#         def __init__(self,line):
#             self.record_type = "BASIC"
#             self.data = line.strip()

#         def __str__(self):
#             return self.data

#         def __repr__self():
#             return str(self)

#     class AtomRecord(BasicRecord):
#         def __init__(self,line):
#             self.record_type = line[0:6].strip()
#             self.serial = int(line[6:11])
#             self.name = line[12:16].strip()
#             self.altLoc = line[16]
#             self.resName = line[17:20].strip()
#             self.chainID = line[21]
#             self.resSeq = int(line[22:26])
#             self.iCode = line[26]
#             self.x = float(line[30:38])
#             self.y = float(line[38:46])
#             self.z = float(line[46:54])
#             self.occupancy = float(line[54:60])
#             self.tempFactor = float(line[60:66])
#             self.element = line[76:78].strip()
#             self.charge = line[78:80].strip()

#         def __str__(self):
#             line = ""

#             line += self.record_type.ljust(6,' ')
#             line += str(self.serial).rjust(5,' ')
#             line += " "
#             line += self.name.ljust(4,' ')
#             line += self.altLoc
#             line += self.resName.rjust(3,' ')
#             line += " "
#             line += self.chainID
#             line += str(self.resSeq).rjust(4,' ')
#             line += self.iCode
#             line += "   "
#             line += f"{self.x:8.3f}"
#             line += f"{self.y:8.3f}"
#             line += f"{self.z:8.3f}"
#             line += f"{self.occupancy:6.2f}"
#             line += f"{self.tempFactor:6.2f}"
#             line += "          "
#             line += self.element.rjust(2,' ')
#             line += self.charge.rjust(2,' ')

#             return line


#         def __repr__self():
#             return str(self)

#     def parse_record(self,line):
#         record_type = line[0:6].strip()

#         if record_type == "ATOM":
#             return self.AtomRecord(line)
#         else:
#             return self.BasicRecord(line)

#     def parse_stream(self,pdb_stream):
#         #clear the main datatype before refilling to prevent accidental appending
#         self.records = []

#         for line in pdb_stream:
#             self.records.append(self.parse_record(line))

#     def parse_file(self,pdb_filename):
#         with open(pdb_filename,'r') as f:
#             self.parse_stream(f)

#     def parse_string(self,pdb_str:str):
#         self.parse_stream(pdb_str.split("\n"))

# The rest of it

In [7]:

def get_cys_positions(pdb_gz):
    positions = set()
    with gzip.open(pdb_gz,'rt') as f:
        for line in f:
            if line.startswith("ATOM") and line[17:20] == "CYS":
                resi = int(line[22:26])
                positions.add(resi)
    return positions
#     decoy = df[df['review_id'] == name]['decoy'].values[0]
#     hbnet_path = hbnet_mapping[decoy]
#     print(hbnet_path)
#     res_i_n_map = {}
#     constraints = []
#     with open(hbnet_path,'r') as pdb:
#         for line in pdb.readlines():
#             if line.startswith("ATOM"):
#                 resname = line[17:20].strip()
#                 resi = int(line[22:26])
#                 res_i_n_map[resi] = resname
#             if line[1:9] == "AtomPair":
#                 constraints.append(line[1:])
    
#     bonds = []
#     for constraint in constraints:
#         #1st is acceptor
#         #2nd is donor

#         #if atom of acceptor is O, that's a bb and I might just ignore.
#         acceptor_atom = constraint.split()[1]
#         acceptor_resi = int(constraint.split()[2])
#         donor_atom = constraint.split()[3]
#         donor_resi = int(constraint.split()[4])
#         if acceptor_atom != "O":
#             bonds.append(((donor_resi,res_i_n_map[donor_resi]),(acceptor_resi,res_i_n_map[acceptor_resi])))
        
        
        
#     #         HBNet_resis = set()

#     #         #convert constraints
#     #         adjusted_constraints = []
#     #         len_chain_A = 66
#     #         for constraint in constraints:
#     #                 constraint_parts = constraint.split()
#     #                 #oops! hardcoded
#     #                 if int(constraint_parts[2]) <= len_chain_A:
#     #                         constraint_parts[2] = f"{constraint_parts[2]}A"
#     #                 else:
#     #                         constraint_parts[2] = f"{int(constraint_parts[2]) + len_chain_A}B"

#     #                 if int(constraint_parts[4]) <= len_chain_A:
#     #                         constraint_parts[4] = f"{constraint_parts[4]}A"
#     #                 else:
#     #                         constraint_parts[4] = f"{int(constraint_parts[4]) + len_chain_A}B"
#     #                 adjusted_constraints.append(" ".join(constraint_parts) + "\n")

#     #         for constraint in adjusted_constraints:
#     #                 HBNet_resis.add(constraint.split()[2])
#     #                 HBNet_resis.add(constraint.split()[4])

#     #         return HBNet_resis
#     return bonds

In [3]:
all_ss_pdbs = glob.glob("output/*.pdb.gz") + glob.glob("manual/*.pdb.gz")

In [84]:
groups = defaultdict(lambda:[])
for ss_pdb in sorted(all_ss_pdbs):
    ss_position = tuple(sorted(get_cys_positions(ss_pdb)))
    groups[ss_position].append(ss_pdb)
    print(ss_pdb,ss_position)

manual/RC_4_20_nmp1_redesign_11_handstaple.pdb.gz (1, 168)
output/RC_4_20_nmp1_redesign_01_staple_0.pdb.gz (17, 184)
output/RC_4_20_nmp1_redesign_01_staple_1.pdb.gz (8, 175)
output/RC_4_20_nmp1_redesign_04_staple_0.pdb.gz (11, 175)
output/RC_4_20_nmp1_redesign_04_staple_1.pdb.gz (11, 175)
output/RC_4_20_nmp1_redesign_04_staple_2.pdb.gz (8, 175)
output/RC_4_20_nmp1_redesign_08_staple_0.pdb.gz (22, 218)
output/RC_4_20_nmp1_redesign_08_staple_1.pdb.gz (22, 218)
output/RC_4_20_nmp1_redesign_08_staple_2.pdb.gz (22, 218)
output/RC_4_20_nmp1_redesign_08_staple_3.pdb.gz (25, 214)
output/RC_4_20_nmp1_redesign_08_staple_4.pdb.gz (22, 218)
output/RC_4_20_nmp1_redesign_09_staple_0.pdb.gz (22, 218)
output/RC_4_20_nmp1_redesign_09_staple_1.pdb.gz (5, 171)
output/RC_4_20_nmp1_redesign_09_staple_2.pdb.gz (25, 214)
output/RC_4_20_nmp1_redesign_13_staple_0.pdb.gz (22, 218)
output/RC_4_20_nmp1_redesign_13_staple_1.pdb.gz (22, 218)
output/RC_4_20_nmp1_redesign_13_staple_2.pdb.gz (12, 178)
output/RC_4_20_n

In [83]:
groups

defaultdict(<function __main__.<lambda>()>,
            {(1, 168): ['manual/RC_4_20_nmp1_redesign_11_handstaple.pdb.gz'],
             (17, 184): ['output/RC_4_20_nmp1_redesign_01_staple_0.pdb.gz'],
             (8, 175): ['output/RC_4_20_nmp1_redesign_01_staple_1.pdb.gz',
              'output/RC_4_20_nmp1_redesign_04_staple_2.pdb.gz',
              'output/RC_4_20_nmp1_redesign_54_staple_0.pdb.gz',
              'output/RC_4_20_nmp1_redesign_71_staple_2.pdb.gz'],
             (11, 175): ['output/RC_4_20_nmp1_redesign_04_staple_0.pdb.gz',
              'output/RC_4_20_nmp1_redesign_04_staple_1.pdb.gz',
              'output/RC_4_20_nmp1_redesign_57_staple_0.pdb.gz',
              'output/RC_4_20_nmp1_redesign_57_staple_1.pdb.gz',
              'output/RC_4_20_nmp1_redesign_57_staple_2.pdb.gz',
              'output/RC_4_20_nmp1_redesign_57_staple_3.pdb.gz',
              'output/RC_4_20_nmp1_redesign_57_staple_5.pdb.gz'],
             (22, 218): ['output/RC_4_20_nmp1_redesign_08_stapl

In [ ]:
#goal: pick the set of designs with the least shared ss positions

# garbage down here is me trying to figure out which is the "best" ss. I gave up

In [37]:
import pyrosetta
#pyrosetta.init('-corrections::beta_nov16 -detect_disulf 1 ')
pyrosetta.init('-corrections::beta_nov16 -detect_disulf 0 ')

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.23+release.0d6f90a8cb9fa0567ca76bb71ee93bfe73340c70 2020-06-04T19:12:24] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r257 2020.23+release.0d6f90a 0d6f90a8cb9fa0567ca76bb71ee93bfe73340c70 http://www.pyrosetta.org 2020-06-04T19:12:24
core.init: {0} command: PyRosetta -corrections::beta_nov16 -detect_disulf 0 -database /home/rdkibler/.conda/envs/pyro/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=1274709738 seed_offset=0 real_seed=1274709738 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode

In [38]:
pose = pyrosetta.pose_from_file(all_ss_pdbs[0])

core.import_pose.import_pose: {0} File 'output/RC_4_20_nmp1_redesign_74_staple_1.pdb.gz' automatically determined to be of type PDB


In [40]:
print(pose)

PDB file name: output/RC_4_20_nmp1_redesign_74_staple_1.pdb.gz
Total residues: 1104
Sequence: DECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQAGISSEEILELLRAAHELGLDPDAIQAAAALGEAGISSEEILELLRAAHELGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGFSPRAIAVAAWFGKLGISSEQITILMLSCAASQLAREADDKDVREIVELALELVSKSKNDEVAKLASRAAYLAARSTDQEVLEEVKEALRRAKESTDEEEIKEELRKAVEEAEDECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQAGISSEEILELLRAAHELGLDPDAIQAAAALGEAGISSEEILELLRAAHELGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGFSPRAIAVAAWFGKLGISSEQITILMLSCAASQLAREADDKDVREIVELALELVSKSKNDEVAKLASRAAYLAARSTDQEVLEEVKEALRRAKESTDEEEIKEELRKAVEEAEDECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQAGISSEEILELLRAAHELGLDPDAIQAAAALGEAGISSEEILELLRAAHELGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGFSPRAIAVAAWFGKLGISSEQITILMLSCAASQLAREADDKDVREIVELALELVSKSKNDEVAKLASRAAYLAARSTDQEVLEEVKEALRRAKESTDEEEIKEELRKAVEEAEDECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQ

In [42]:
disulfide_entropy = pyrosetta.rosetta.protocols.simple_filters.DisulfideEntropyFilter()

In [43]:
disulfide_entropy.compute(pose)

0.0

In [44]:
pose.conformation().detect_disulfides()

In [45]:
disulfide_entropy.compute(pose)

0.0

In [27]:
#pyrosetta.rosetta.protocols.symmetry.DetectSymmetry().apply(pose)

In [29]:
scom = pyrosetta.rosetta.protocols.simple_moves.SwitchChainOrderMover()
scom.chain_order("AB")


core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: beta_nov16.wts
core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.io.database: {0}

In [30]:
scom.apply(pose)

protocols.simple_moves.SwitchChainOrderMover: {0} Number of chains in pose: 4
protocols.simple_moves.SwitchChainOrderMover: {0} Now at chain: 1
protocols.simple_moves.SwitchChainOrderMover: {0} Now at chain: 2
core.conformation.Conformation: {0} Reverting out-of-date disulfide to thiol type at resid 3
core.conformation.Conformation: {0} Reverting out-of-date disulfide to thiol type at resid 192
core.conformation.Conformation: {0} Reverting out-of-date disulfide to thiol type at resid 279
core.conformation.Conformation: {0} Reverting out-of-date disulfide to thiol type at resid 468
basic.io.database: {0} Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: {0} Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: {0} shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: {0} Binary rotamer library selected: /home/rdkibler/.conda/envs/pyro/lib/python3.7/site-packages/pyrosetta/database/rotamer/beta_nov2016/Dunb

In [62]:
force_disulfides = pyrosetta.rosetta.protocols.calc_taskop_movers.ForceDisulfidesMover()
disulfides_list = pyrosetta.rosetta.utility.vector1_std_pair_unsigned_long_unsigned_long_t()
disulfides_list.append(tuple(get_cys_positions(all_ss_pdbs[0])))
force_disulfides.disulfides(disulfides_list)

In [64]:
force_disulfides.apply(pose)

protocols.simple_moves.ForceDisulfidesMover: {0} Fixing disulfides
core.pack.task: {0} Packer task: initialize from command line()
protocols.simple_moves.ForceDisulfidesMover: {0} repacking disulfide surroundings
protocols.minimization_packing.PackRotamersMover: {0} [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: beta_nov16.wts
core.pack.pack_rotamers: {0} built 107 rotamers at 7 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
basic.thread_manager.RosettaThreadManager: {?} Creating a thread pool of 1 threads.
basic.thread_manager.RosettaThreadPool: {?} Launched 0 new threads.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).


In [66]:
print(pose)

PDB file name: output/RC_4_20_nmp1_redesign_74_staple_1.pdb.gz
Total residues: 1104
Sequence: DECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQAGISSEEILELLRAAHELGLDPDAIQAAAALGEAGISSEEILELLRAAHELGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGFSPRAIAVAAWFGKLGISSEQITILMLSCAASQLAREADDKDVREIVELALELVSKSKNDEVAKLASRAAYLAARSTDQEVLEEVKEALRRAKESTDEEEIKEELRKAVEEAEDECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQAGISSEEILELLRAAHELGLDPDAIQAAAALGEAGISSEEILELLRAAHELGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGFSPRAIAVAAWFGKLGISSEQITILMLSCAASQLAREADDKDVREIVELALELVSKSKNDEVAKLASRAAYLAARSTDQEVLEEVKEALRRAKESTDEEEIKEELRKAVEEAEDECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQAGISSEEILELLRAAHELGLDPDAIQAAAALGEAGISSEEILELLRAAHELGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGFSPRAIAVAAWFGKLGISSEQITILMLSCAASQLAREADDKDVREIVELALELVSKSKNDEVAKLASRAAYLAARSTDQEVLEEVKEALRRAKESTDEEEIKEELRKAVEEAEDECLEAAAALGQAGIDAKEINELLLAIQQLGLDPDAIQAAAQLGEAGISSEEILELLRAAHELGLDPDAIAAAADLGQ

In [67]:
pose.dump_pdb("test.pdb")

True

In [47]:
force_disulfides.disulfides(pose)

vector1_std_pair_unsigned_long_unsigned_long_t[]

In [21]:
get_cys_positions(all_ss_pdbs[0])

{3, 192}

In [55]:
pyrosetta.Vector1(tuple)

TypeError: 'type' object is not iterable

0.0

In [25]:
disulfide_entropy.compute(pose)

0.0

In [75]:
ss_pair = tuple(sorted(get_cys_positions(all_ss_pdbs[0])))
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects().create_from_string(f"""
<MOVERS>
    <ForceDisulfides name="force_ss" disulfides="{ss_pair[0]}B:{ss_pair[1]}A" repack="true"/>
</MOVERS>
""")
force_ss = xml.get_mover("force_ss")

protocols.rosetta_scripts.RosettaScriptsParser: {0} Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: {0} ...done
protocols.rosetta_scripts.RosettaScriptsParser: {0} Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: {0} ...done
protocols.rosetta_scripts.RosettaScriptsParser: {0} Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: {0} ...done
protocols.rosetta_scripts.RosettaScriptsParser: {0} Parsed script:
<ROSETTASCRIPTS>
	<MOVERS>
		<ForceDisulfides disulfides="3B:192A" name="force_ss" repack="true"/>
	</MOVERS>
	<PROTOCOLS/>
</ROSETTASCRIPTS>
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: beta_nov16.wts
protocols.simple_moves.ForceDisulfidesMover: {0} Setting fix disulfides on residues:
protocols.simple_moves.ForceDisulfidesMover: {0}         Residue 3  chain B : Residue 192  chain A
protocols.rosetta_scripts.RosettaScriptsParser: {0} Defined mover named "force_ss" of type ForceD

In [78]:
pose = pyrosetta.pose_from_file(all_ss_pdbs[0])
pyrosetta.rosetta.protocols.symmetry.DetectSymmetry().apply(pose)

core.import_pose.import_pose: {0} File 'output/RC_4_20_nmp1_redesign_74_staple_1.pdb.gz' automatically determined to be of type PDB
protocols.simple_moves.symmetry.DetectSymmetry: {0} 4 number of subunits found
protocols.simple_moves.symmetry.DetectSymmetry: {0} 276 residues per subunit
core.conformation.symmetry.SymmData: {0} [ WARNING ] Setting weight of master jump ( jump-id=8 ) to 1.0 (was undefined)
core.conformation.symmetry.SymmData: {0} [ WARNING ] Setting jump_group JGS: [master 8]  9:0  10:0  11:0
core.conformation.symmetry.SymmData: {0} [ WARNING ] Setting weight of master jump ( jump-id=1 ) to 1.0 (was undefined)
core.conformation.symmetry.SymmData: {0} [ WARNING ] Setting jump_group JGSS: [master 1]  2:0  3:0  4:0
core.conformation.symmetry.SymmData: {0} symmetry name: C4
core.conformation.symmetry.SymmData: {0} number of subunits: 4
core.conformation.symmetry.SymmData: {0} number of interfaces: 3
core.conformation.symmetry.SymmData: {0} score subunit number: C1
core.confo

In [80]:
force_ss.apply(pose)

protocols.simple_moves.ForceDisulfidesMover: {0} Fixing disulfides
core.pack.task: {0} Packer task: initialize from command line()
protocols.simple_moves.ForceDisulfidesMover: {0} repacking disulfide surroundings


RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/scoring/disulfides/FullatomDisulfideEnergyContainer.cc:619
[ ERROR ] UtilityExitException
ERROR: 

